# Importing necessary libraries


In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_log_error
import math
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

# Loading the data files

In [45]:
houses_pred = pd.read_csv('train.csv')
houses_pred.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
houses_pred.info()

In [46]:
houses_pred.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

# Feature selection

In [47]:
filtered_df = houses_pred[['MSSubClass', 'MSZoning','Utilities', 'OverallQual', 'CentralAir', 'OverallCond', 'GarageArea', 'BldgType', 'SalePrice']]

In [48]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MSSubClass   1460 non-null   int64 
 1   MSZoning     1460 non-null   object
 2   Utilities    1460 non-null   object
 3   OverallQual  1460 non-null   int64 
 4   CentralAir   1460 non-null   object
 5   OverallCond  1460 non-null   int64 
 6   GarageArea   1460 non-null   int64 
 7   BldgType     1460 non-null   object
 8   SalePrice    1460 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 102.8+ KB


In [49]:
filtered_df.isnull().sum()

MSSubClass     0
MSZoning       0
Utilities      0
OverallQual    0
CentralAir     0
OverallCond    0
GarageArea     0
BldgType       0
SalePrice      0
dtype: int64

# Feature processing

In [50]:
filtered_df.columns


Index(['MSSubClass', 'MSZoning', 'Utilities', 'OverallQual', 'CentralAir',
       'OverallCond', 'GarageArea', 'BldgType', 'SalePrice'],
      dtype='object')

In [51]:

 
mmscaler = MinMaxScaler()
cols = ['GarageArea']
filtered_df[cols] = mmscaler.fit_transform(filtered_df[cols])


/Users/suryatejasista/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [52]:
filtered_df['GarageArea']

0       0.386460
1       0.324401
2       0.428773
3       0.452750
4       0.589563
          ...   
1455    0.324401
1456    0.352609
1457    0.177715
1458    0.169252
1459    0.194640
Name: GarageArea, Length: 1460, dtype: float64

In [53]:
filtered_df['Utilities'].value_counts()

AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64

In [54]:
filtered_df['MSZoning'].value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

In [55]:
filtered_df['BldgType'].value_counts()

1Fam      1220
TwnhsE     114
Duplex      52
Twnhs       43
2fmCon      31
Name: BldgType, dtype: int64

In [56]:
filtered_df['CentralAir'].value_counts()

Y    1365
N      95
Name: CentralAir, dtype: int64

### Label_encoding

In [57]:
filtered_df.replace({'BldgType':{'1Fam':1, '2fmCon': 2, 'TwnhsE':3, 'Duplex':4, 'Twnhs':5}}, inplace = True)

/Users/suryatejasista/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [58]:
filtered_df['BldgType'].value_counts()

1    1220
3     114
4      52
5      43
2      31
Name: BldgType, dtype: int64

In [59]:
filtered_df.replace({'Utilities':{'AllPub':1, 'NoSeWa': 2}}, inplace = True)
filtered_df['Utilities'].value_counts()

1    1459
2       1
Name: Utilities, dtype: int64

In [60]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder_df = pd.DataFrame(encoder.fit_transform(filtered_df[['MSZoning']]).toarray())
filtered_df = filtered_df.join(encoder_df)
filtered_df

,MSSubClass,MSZoning,Utilities,OverallQual,CentralAir,OverallCond,GarageArea,BldgType,SalePrice,0,1,2,3,4
0,60,RL,1,7,Y,5,0.386460,1,208500,0.0,0.0,0.0,1.0,0.0
1,20,RL,1,6,Y,8,0.324401,1,181500,0.0,0.0,0.0,1.0,0.0
2,60,RL,1,7,Y,5,0.428773,1,223500,0.0,0.0,0.0,1.0,0.0
3,70,RL,1,7,Y,5,0.452750,1,140000,0.0,0.0,0.0,1.0,0.0
4,60,RL,1,8,Y,5,0.589563,1,250000,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,1,6,Y,5,0.324401,1,175000,0.0,0.0,0.0,1.0,0.0
1456,20,RL,1,6,Y,6,0.352609,1,210000,0.0,0.0,0.0,1.0,0.0
1457,70,RL,1,7,Y,9,0.177715,1,266500,0.0,0.0,0.0,1.0,0.0
1458,20,RL,1,5,Y,6,0.169252,1,142125,0.0,0.0,0.0,1.0,0.0


In [61]:
filtered_df['MSZoning'].value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

In [62]:
filtered_df.drop('MSZoning', axis=1, inplace=True)

In [63]:
filtered_df.columns

Index([ 'MSSubClass',   'Utilities', 'OverallQual',  'CentralAir',
       'OverallCond',  'GarageArea',    'BldgType',   'SalePrice',
                   0,             1,             2,             3,
                   4],
      dtype='object')

In [64]:

filtered_df.columns = ['MSSubClass',   'Utilities', 'OverallQual',  'CentralAir',
       'OverallCond',  'GarageArea',    'BldgType', 'SalePrice', 'RL', 'RM', 'FV', 'RH', 'C (all)']

In [65]:
filtered_df.head(5)

,MSSubClass,Utilities,OverallQual,CentralAir,OverallCond,GarageArea,BldgType,SalePrice,RL,RM,FV,RH,C (all)
0,60,1,7,Y,5,0.386460,1,208500,0.0,0.0,0.0,1.0,0.0
1,20,1,6,Y,8,0.324401,1,181500,0.0,0.0,0.0,1.0,0.0
2,60,1,7,Y,5,0.428773,1,223500,0.0,0.0,0.0,1.0,0.0
3,70,1,7,Y,5,0.452750,1,140000,0.0,0.0,0.0,1.0,0.0
4,60,1,8,Y,5,0.589563,1,250000,0.0,0.0,0.0,1.0,0.0


In [66]:
filtered_df.replace({'CentralAir':{'Y':1, 'N': 2}}, inplace = True)

In [67]:
filtered_df.head(5)

,MSSubClass,Utilities,OverallQual,CentralAir,OverallCond,GarageArea,BldgType,SalePrice,RL,RM,FV,RH,C (all)
0,60,1,7,1,5,0.386460,1,208500,0.0,0.0,0.0,1.0,0.0
1,20,1,6,1,8,0.324401,1,181500,0.0,0.0,0.0,1.0,0.0
2,60,1,7,1,5,0.428773,1,223500,0.0,0.0,0.0,1.0,0.0
3,70,1,7,1,5,0.452750,1,140000,0.0,0.0,0.0,1.0,0.0
4,60,1,8,1,5,0.589563,1,250000,0.0,0.0,0.0,1.0,0.0


In [68]:
X = filtered_df[['MSSubClass',   'Utilities', 'OverallQual',  'CentralAir',
       'OverallCond',  'GarageArea',    'BldgType', 'RL', 'RM', 'FV', 'RH', 'C (all)']]
X.head(5)

,MSSubClass,Utilities,OverallQual,CentralAir,OverallCond,GarageArea,BldgType,RL,RM,FV,RH,C (all)
0,60,1,7,1,5,0.386460,1,0.0,0.0,0.0,1.0,0.0
1,20,1,6,1,8,0.324401,1,0.0,0.0,0.0,1.0,0.0
2,60,1,7,1,5,0.428773,1,0.0,0.0,0.0,1.0,0.0
3,70,1,7,1,5,0.452750,1,0.0,0.0,0.0,1.0,0.0
4,60,1,8,1,5,0.589563,1,0.0,0.0,0.0,1.0,0.0


# Model training

In [69]:
X.head()

,MSSubClass,Utilities,OverallQual,CentralAir,OverallCond,GarageArea,BldgType,RL,RM,FV,RH,C (all)
0,60,1,7,1,5,0.386460,1,0.0,0.0,0.0,1.0,0.0
1,20,1,6,1,8,0.324401,1,0.0,0.0,0.0,1.0,0.0
2,60,1,7,1,5,0.428773,1,0.0,0.0,0.0,1.0,0.0
3,70,1,7,1,5,0.452750,1,0.0,0.0,0.0,1.0,0.0
4,60,1,8,1,5,0.589563,1,0.0,0.0,0.0,1.0,0.0


In [70]:
filtered_df.columns

Index(['MSSubClass', 'Utilities', 'OverallQual', 'CentralAir', 'OverallCond',
       'GarageArea', 'BldgType', 'SalePrice', 'RL', 'RM', 'FV', 'RH',
       'C (all)'],
      dtype='object')

In [71]:
y = filtered_df['SalePrice']
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

### Splitting the data into training and testing data

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [73]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1168, 12)
X_test shape: (292, 12)
y_train shape: (1168,)
y_test shape: (292,)


In [74]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

# Model evaluation

In [75]:
y_pred = lin_reg.predict(X_test)

### using root mean sqaure error method

In [76]:
def compute_rmsle(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
    return round(rmsle, precision)

compute_rmsle(y_test, y_pred)

0.24